<a href="https://colab.research.google.com/github/kmk4444/-Calculating-potential-customer-revenue-using-rule-based-classification/blob/main/Haber_cekme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install datetime

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from openai import OpenAI  # OpenAI API için gerekli
import base64
import requests
from email.mime.image import MIMEImage

# OpenAI API anahtarını ayarla
client = OpenAI(
    # This is the default and can be omitted
    api_key='sk-proj-',
)

# SerpAPI anahtarını ayarla
SERP_API_KEY = ""


# Haber özetinin ilgili olup olmadığını kontrol eden fonksiyon
def is_relevant(summary):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{
                "role": "system",
                "content": "Sen yardımcı bir asistanısın ve verilen metnin Aselsan veya savunma sanayii ile ilgili olup olmadığını belirliyorsun."
            }, {
                "role": "user",
                "content": f"Aşağıdaki metin Aselsan veya savunma sanayii ile ilgili mi? Lütfen sadece 'Evet' veya 'Hayır' şeklinde cevap ver.\n\n{summary}"
            }],
            max_tokens=5,
            temperature=0
        )
        if response and response.choices:
            answer = response.choices[0].message.content.strip().lower()
            if 'evet' in answer:
                return True
            else:
                return False
        else:
            return False
    except Exception as e:
        print(f"İlgililik kontrolü sırasında hata oluştu: {e}")
        return False




# This function encodes the image to base64
def get_image_base64(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            return base64.b64encode(response.content).decode('utf-8')
        else:
            return None
    except Exception as e:
        print(f"Error fetching image from {image_url}: {e}")
        return None

# Google'da haber aramak için SerpAPI kullan
def search_aselsan_news(start_date, end_date):
    params = {
        "q": "Aselsan haberleri OR Aselsan news",
        "tbm": "nws",
        "tbs": f"cdr:1,cd_min:{start_date},cd_max:{end_date}",
        "api_key": SERP_API_KEY
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    results = response.json()

    print("SerpAPI Yanıtı:", response.json())

    news_links = []
    if 'news_results' in results:
        for news in results['news_results']:
            news_links.append({
                'title': news['title'],
                'link': news['link'],
                'date': news['date'],
                'source': news['source'],  # Haber kaynağı eklendi
                'thumbnail': news['thumbnail']
            })
    return news_links

# Web sayfasından haber içeriğini çek
def scrape_news_content(news_url):
    try:
        response = requests.get(news_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            content = ' '.join([p.get_text() for p in paragraphs])
            return content
        else:
            return None
    except Exception as e:
        print(f"Error fetching {news_url}: {e}")
        return None

# OpenAI API ile haber içeriğine tavsiye ver (gpt-4 modelini kullanarak)
def ai_advice(content):
    try:
        # OpenAI API'yi kullanarak özetleme yapıyoruz
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{
                "role": "system",
                "content": """Sen yardımsever bir Savunma Sanayii Uzmanısın. Senin verdiğin bu tavsiye şirketimin geleceği açısından çok önemlidir. Stratejik tavsiyeni yöneticilerime vereceğim. Bu nedenle siyasi, politik, cinsellik ve benzeri hassas konular içeren her şeyden uzak durmalısın. //Similar news// veya //benzer haberler// yazısı var ise sonraki cümleleri dahil etmeden tavsiye vermelisin.

                Örneğin: Güney Afrika merkezli DCD Protected Mobility, Africa Aerospace and Defence (AAD) 2024 fuarında yeni bir zırhlı araç modeli olan Springbuck Standard Duty (SD)’yi tanıttı. Çeşitli silah sistemleriyle entegre edilebilen bu model, ASELSAN tarafından geliştirilen Alkar 120 mm havan sistemi ile sergilendi. Springbuck SD, çeşitli operasyonel ortamlarda koruma ve hareket kabiliyeti arasında denge sağlamak üzere tasarlanmış bir mayın dirençli araçtır. Araç, küçük silah ateşine direnç sağlayan EN 1522 Level B6 seviyesinde zırha sahiptir. Araç, 2+2 konfigürasyonunda beş kişiye kadar oturma ve ek bir ayakta silahşör pozisyonu sunmaktadır. Alkar 120 mm Havancı Silah Sistemi, ASELSAN tarafından orijinal olarak tasarlanmış modern bir silah sistemidir. Otomatik namlu yönlendirmesi, otomatik mühimmat yükleme, geri tepme mekanizması ve ateş kontrol sistemleri ile donatılmıştır. Bu sistem, hızlı atış yeteneği ve gelişmiş hedefleme sistemleri ile 8.000 metreye kadar etkili bir menzil sunar.
                //Benzer Haberler veya //Similar News//, Ulaştırma ve Altyapı Bakanlığı tarafından başlatılan Kuzey Hafif Raylı Sistem Hattı çalışmaları, Mannesman bölgesindeki SEKA Park-İzmit durağından sonra Derince Millet Bahçesi durağında başladı. DEM Partisi, ilk yıldönümünü kutluyor! Barış ve demokrasi vurgusuyla yola çıkan parti, toplumsal mutabakatı hedefliyor. Geçmiş başarılarını değerlendirirken, gelecek için umutlu hedefler belirliyor. TRT Genel Müdürü Sobacı'nın son açıklamaları, medya dünyasında geniş yankı buldu. Sobacı'nın sözlerinin kamuoyunu nasıl etkilediğini öğrenin. Devrim Muhafızları Komutanı Abbas Nilfurushan'ın cenaze töreni, askeri törenler ve halkın katılımıyla Tahran'da gerçekleştirildi. Tören, ulusal yas ve anma ile doluydu. İsrail ve ABD'nin İran'a karşı askeri eylem planlarına derinlemesine bir bakış atıyoruz. Bu stratejik ilişkilerin arka planı, hedefleri ve potansiyel sonuçları hakkında bilgi edinin. Aydın-Denizli Otoyolu'nun açılış tarihi resmi olarak açıklandı.
                Perşembe günü 17 Ekim 00.01'den itibaren trafiğe açılacak.

                Yukarıdaki örnekte göreceğin üzere //Benzer Haberler veya //Similar News// kısmı ve hassas konular bulunmaktadır. Bu kısımları silmelisin.

                """
            }, {
                "role": "user",
                "content": f"""Aşağıdaki yazıyı anlayıp, Savunma Sanayii yöneticileri için Türkçe ve maksimum 75 token olacak şekilde stratejik tavsiye verebilir misin? Senin verdiğin bu tavsiye şirketimin geleceği açısından çok önemlidir. Stratejik tavsiyeni yöneticilerime vereceğim. Bu nedenle siyasi, politik, cinsellik ve benzeri hassas konular içeren her şeyden uzak durmalısın.  //Similar news// veya //benzer haberler// yazısı var ise sonraki cümleleri dahil etmeden tavsiye vermelisin.


                \n\n{content}"""
            }],
            max_tokens=1000,
            temperature=0.7
        )
        # Yanıtı kontrol edelim
        if response and response.choices:
            summary = response.choices[0].message.content.strip()  # Hatanın düzeltildiği yer
            return summary
        else:
            return "Özet oluşturulamadı."
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Özetleme sırasında bir hata oluştu."




# OpenAI API ile haber içeriğini özetle (gpt-4 modelini kullanarak)
def summarize_content(content):
    try:
        # OpenAI API'yi kullanarak özetleme yapıyoruz
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{
                "role": "system",
                "content": """Sen yardımsever çeviri ve özet uzmanısın. Senin verdiğin maksimum 150 token olan özet yazısı şirketimin geleceği açısından çok önemlidir. Özet yazını yöneticilerime vereceğim. Bu nedenle siyasi, politik, cinsellik ve benzeri hassas konular içeren her şeyden uzak durmalısın. Ayrıca, //Similar news// veya //benzer haberler// yazısı var ise sonraki cümleleri dahil etmeden tavsiye vermelisin.

                Örneğin: Güney Afrika merkezli DCD Protected Mobility, Africa Aerospace and Defence (AAD) 2024 fuarında yeni bir zırhlı araç modeli olan Springbuck Standard Duty (SD)’yi tanıttı. Çeşitli silah sistemleriyle entegre edilebilen bu model, ASELSAN tarafından geliştirilen Alkar 120 mm havan sistemi ile sergilendi. Springbuck SD, çeşitli operasyonel ortamlarda koruma ve hareket kabiliyeti arasında denge sağlamak üzere tasarlanmış bir mayın dirençli araçtır. Araç, küçük silah ateşine direnç sağlayan EN 1522 Level B6 seviyesinde zırha sahiptir. Araç, 2+2 konfigürasyonunda beş kişiye kadar oturma ve ek bir ayakta silahşör pozisyonu sunmaktadır. Alkar 120 mm Havancı Silah Sistemi, ASELSAN tarafından orijinal olarak tasarlanmış modern bir silah sistemidir. Otomatik namlu yönlendirmesi, otomatik mühimmat yükleme, geri tepme mekanizması ve ateş kontrol sistemleri ile donatılmıştır. Bu sistem, hızlı atış yeteneği ve gelişmiş hedefleme sistemleri ile 8.000 metreye kadar etkili bir menzil sunar.
                //Benzer Haberler veya //Similar News//, Ulaştırma ve Altyapı Bakanlığı tarafından başlatılan Kuzey Hafif Raylı Sistem Hattı çalışmaları, Mannesman bölgesindeki SEKA Park-İzmit durağından sonra Derince Millet Bahçesi durağında başladı. DEM Partisi, ilk yıldönümünü kutluyor! Barış ve demokrasi vurgusuyla yola çıkan parti, toplumsal mutabakatı hedefliyor. Geçmiş başarılarını değerlendirirken, gelecek için umutlu hedefler belirliyor. TRT Genel Müdürü Sobacı'nın son açıklamaları, medya dünyasında geniş yankı buldu. Sobacı'nın sözlerinin kamuoyunu nasıl etkilediğini öğrenin. Devrim Muhafızları Komutanı Abbas Nilfurushan'ın cenaze töreni, askeri törenler ve halkın katılımıyla Tahran'da gerçekleştirildi. Tören, ulusal yas ve anma ile doluydu. İsrail ve ABD'nin İran'a karşı askeri eylem planlarına derinlemesine bir bakış atıyoruz. Bu stratejik ilişkilerin arka planı, hedefleri ve potansiyel sonuçları hakkında bilgi edinin. Aydın-Denizli Otoyolu'nun açılış tarihi resmi olarak açıklandı.
                Perşembe günü 17 Ekim 00.01'den itibaren trafiğe açılacak.

                Yukarıdaki örnekte göreceğin üzere //Benzer Haberler veya //Similar News// kısmı ve hassas konular bulunmaktadır. Bu kısımları silmelisin.

                """
            }, {
                "role": "user",
                "content": f"Aşağıdaki yazıyı Türkçe ve maksimum 150 token olacak şekilde resmi bir yazı dilinde ve açıklayıcı özetler misin? Bu özeti yöneticilerime ileteceğim için siyasi, politik, cinsellik ve benzeri hassas konular içeren her şeyden uzak durmalısın. //Similar news// veya //benzer haberler// yazısı var ise sonraki cümleleri dahil etmeden tavsiye vermelisin. \n\n{content}"
            }],
            max_tokens=2000,
            temperature=0.5
        )
        # Yanıtı kontrol edelim
        if response and response.choices:
            summary = response.choices[0].message.content.strip()  # Hatanın düzeltildiği yer
            return summary
        else:
            return "Özet oluşturulamadı."
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Özetleme sırasında bir hata oluştu."

# Raporu CSV formatında kaydet (sütunları ";" ile ayır)
def save_news_to_csv(news_data):
    df = pd.DataFrame(news_data)
    file_name = f"aselsan_news_report_{datetime.now().strftime('%Y-%m-%d')}.csv"
    df.to_csv(file_name, index=False, encoding='utf-8', sep=';')
    print(f"Rapor oluşturuldu: {file_name}")



def create_eml_file(news_data, start_date, end_date):
    # Mesajı 'related' olarak tanımlayın
    msg = MIMEMultipart('related')
    msg['From'] = 'ASELNEWS'
    msg['To'] = 'YapayZekaveTeknolojilerBirimi'
    msg['Subject'] = f"ASELSAN Bülteni ({start_date} - {end_date})"

    # Alternatif içerik oluşturun (HTML ve düz metin)
    msg_alternative = MIMEMultipart('alternative')
    msg.attach(msg_alternative)

    # HTML içeriği
    body = f"<h1>ASELSAN Bülteni</h1>"
    body += f"<p><strong>Tarih Aralığı:</strong> {start_date} - {end_date}</p>"

    for idx, news in enumerate(news_data):
        body += f"<h2>{news['Title']}</h2>"
        body += f"<p><strong>Kaynak:</strong> {news['Source']}</p>"

        # Resmi ek olarak ekleyip Content-ID ile referans verin
        img_data = get_image_base64(news['Thumbnail'])
        if img_data:
            img_filename = f"image_{idx}.jpg"
            image_cid = f"image_cid_{idx}"

            # HTML içinde resmi 'cid' ile referans verin
            body += f"<img src='cid:{image_cid}' alt='Haber Resmi' style='max-width:100%;'><br>"

            # Resmi MIMEImage ile ek olarak ekleyin
            image_part = MIMEImage(base64.b64decode(img_data), name=img_filename)
            image_part.add_header('Content-ID', f'<{image_cid}>')
            image_part.add_header('Content-Disposition', 'inline', filename=img_filename)
            msg.attach(image_part)

        body += f"<p>{news['Summary']}</p>"
        body += f"<a href='{news['Link']}'>Haberi Oku</a><br>"
        body += f"<p><strong>Yapay Zeka Stratejik Tavsiyesi:</strong> {news['YZ Tavsiye']}</p>"
        body += f"<p><strong>Yayınlanma Tarihi:</strong> {news['Date']}</p><hr>"

    # HTML içeriğini alternatif kısma ekleyin
    msg_alternative.attach(MIMEText(body, 'html'))

    # .eml dosyası olarak kaydedin
    eml_file_name = f"aselsan_bulteni_{datetime.now().strftime('%Y-%m-%d')}.eml"
    with open(eml_file_name, 'w') as f:
        f.write(msg.as_string())

    print(f"E-posta oluşturuldu ve kaydedildi: {eml_file_name}")



# Ana fonksiyon
def get_aselsan_news_report(start_date, end_date):
    news_links = search_aselsan_news(start_date, end_date)

    if news_links:
        news_data = []
        for news in news_links:
            print(f"Processing: {news['title']}")
            content = scrape_news_content(news['link'])
            if content:
                summary = summarize_content(content)  # OpenAI API ile özet
                # İlgililik kontrolü
                if is_relevant(summary):
                    yz_tavsiye = ai_advice(content)  # OpenAI API ile tavsiye
                    news_data.append({
                        'Title': news['title'],
                        'Link': news['link'],
                        'Date': news['date'],
                        'Source': news['source'],  # Kaynak bilgisi eklendi
                        'Summary': summary,
                        'Thumbnail': news['thumbnail'],  # Haber resmi eklendi
                        'YZ Tavsiye': yz_tavsiye  # Öneri eklendi
                    })
                else:
                    print(f"İlgisiz haber atlandı: {news['title']}")
        if news_data:
            save_news_to_csv(news_data)
            create_eml_file(news_data, start_date, end_date)  # EML dosyası oluşturma
        else:
            print("İlgili haber bulunamadı.")
    else:
        print("Belirtilen tarih aralığında haber bulunamadı.")


# Tarih aralığını belirtin
start_date = "10/1/2024"
end_date = "10/13/2024"

# Ana fonksiyonu çalıştır
get_aselsan_news_report(start_date, end_date)
